# Initial Setup

In [1]:
from bs4 import Tag, NavigableString, BeautifulSoup
import os
from datetime import datetime
from natsort import os_sorted
import pandas as pd
from IPython.display import display, HTML
import matplotlib as plt
import matplotlib.dates as mdates
import string
from wordcloud import WordCloud, STOPWORDS

In [2]:
# Single file: create a file list sorted by first to last message

#folder = "c:/Users/DANKINAH/Downloads/Personal/Messages/"
folder = "c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/"
#folder = "c:/Users/Daniel/Documents/Archive/Facebook Data/messages/inbox/GARGBOYZ_uNSTploxvw/"
members = ['Robby Royston', 'Erik Porteous', 'Nick Cornell', 'Tristan Turigan', 'Daniel Kinahan', 'Alex Staszak', 'Ben Wood', 'Caelan Rae-Oulette', 'Theo Mohamed', 'Francois Jolicoeur']
file_list = []

for filename in os.listdir(folder):
    if filename.endswith('.html'):
       fname = os.path.join(folder, filename)
       file_list.append(fname)

file_list = os_sorted(file_list)
file_list.reverse()
print(file_list)

['c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_18.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_17.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_16.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_15.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_14.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_13.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_12.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_11.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_10.html', 'c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messa

In [3]:
# Multi-file: Create a file list

folder = "c:/Users/DANKINAH/Downloads/Personal/Chats/"
#folder = "c:/Users/Daniel/Documents/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/"
file_list = []

for subfolder in os.scandir(folder):
    cur_file_list = []
    for filename in os.listdir(subfolder.path):
        if filename.endswith('.html'):
            fname = os.path.join(subfolder.path, filename)
            cur_file_list.append(fname)
    cur_file_list = os_sorted(cur_file_list)
    cur_file_list.reverse()
    file_list += cur_file_list
display(file_list)

c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_18.html 7506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_17.html 17506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_16.html 27506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_15.html 37506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_14.html 47506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_13.html 57506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_12.html 67506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_11.html 77506
c:/Users/Daniel/Nextcloud/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/message_10.html 87506
c:/Users/Daniel/Nextcloud/Arc

In [ ]:
nickname_strings = ('set the nickname for', 'set your nickname', 'set his own nickname', 'cleared the nickname for', 'cleared your nickname')
group_strings = ('the group.', 'the group photo.', 'set the emoji to', 'named the group')
misc_strings = ('points playing basketball.', 'Plan created:')

In [ ]:
# Grab messages from files and place in a sorted list

message_list = []

for filename in file_list:
	with open(filename, encoding='utf-8') as f:
		soup = BeautifulSoup(f, 'html.parser')
	cur_message_list = []
	messages = soup.find_all(class_='pam _3-95 _2pi0 _2lej uiBoxWhite noborder')
	for message in messages[1:]:
		#Types of messages: text with embeds, one or many media items
		sender = message.contents[0].string

		content_text = ""
		content_media = {
			'images':[],
			'videos':[],
			'audio':[],
			'reactions':[]
		}

		if len(message.contents[1].contents[0]) == 4: #Regular message
			for item in message.contents[1].contents[0].contents[1].contents:
				if isinstance(item, Tag):
					if item.name == 'br':
						content_text += '\n'
					elif item.name=='a':
						content_text += item['href']
					elif item.name=='img':
						content_text += item['src']
						content_media['images'].append(item['src'])
					elif item.name=='span': #Arabic text
						content_text += item.string
					else:
						print("Unknown embedded item name: " + item.name)
						exit(1)

				elif isinstance(item, NavigableString):
					content_text += item
				else:
					print("Unknown object type in content: " + str(type(item)))		

		elif len(message.contents[1].contents[0]) > 4: #One or more media
			for item in message.contents[1].contents[0].contents:
				if len(item.contents) > 0:
					if item.find('img'):
						content_media['images'].append(item.find('img')['src'])
					elif item.find('video'):
						content_media['videos'].append(item.find('video')['src'])
					elif item.find('audio'):
						content_media['audio'].append(item.find('audio')['src'])
					elif item.find('ul'):
						for reaction in item.find('ul').contents:
							content_media['reactions'].append(reaction.string)

					elif isinstance(item.string, NavigableString): #attached message
						content_text += item.string
					elif isinstance(item, Tag): #attached message with embeds
						for atom in item.contents:
							if isinstance(atom, Tag):
								if atom.name == 'br':
									content_text += '\n'
								else:
									print("Unknown embedded item name (multi-media): " + str(atom.name))
									print(atom)
									exit(1)
							elif isinstance(atom, NavigableString):
								content_text += atom
							else:
								print("Unknown object type in content (multi-media): " + str(type(atom)))	
					else:
						print("Unknown content item: " + str(type(item)))
						exit(1)

		timestamp = datetime.strptime(str(message.contents[2].string), '%d %b %Y, %H:%M')

		if any(x in content_text for x in nickname_strings):
			message_type = 'nickname'
		elif any(x in content_text for x in group_strings):
			message_type = 'group'
		elif any(x in content_text for x in misc_strings):
			message_type = 'misc'
		else:
			message_type = 'regular'
		
		message_dict = {
			'sender': sender, 
			'content_text':content_text, 
			'content_media':content_media, 
			'timestamp': timestamp,
			'message_type': message_type
		}
		cur_message_list.append(message_dict)

	cur_message_list.reverse()
	message_list += cur_message_list
	print(filename, len(message_list))

# Individual reports

In [ ]:
#Print nicknames for each member

nicknames = dict()

for message_dict in message_list:
	content_text = message_dict['content_text']
	message_type = message_dict['message_type']
	if message_type != 'nickname':
		continue
	if nickname_strings[0] in content_text:
		nickname = content_text.split(nickname_strings[0])[1].split(' to ')[1][:-1]
		name = content_text.split(nickname_strings[0])[1].split(' to ')[0][1:]
	elif nickname_strings[1] in content_text or nickname_strings[2] in content_text:
		nickname = content_text.split('nickname')[1].split(' to ')[1][:-1]
		name = message_dict['sender']

	if name not in nicknames:
		nicknames[name] = []
	else:
		nicknames[name].append(nickname)

print('Nicknames of each member')
for name in nicknames.keys():
    print(name + ": " + ', '.join(nicknames[name]) + "\n")

In [ ]:
#Print nicknames for each member grouped by day

nicknames = []

for message_dict in message_list:
	content_text = message_dict['content_text']
	message_type = message_dict['message_type']
	if message_type != 'nickname':
		continue
	if nickname_strings[0] in content_text:
		nickname = content_text.split(nickname_strings[0])[1].split(' to ')[1][:-1]
		name = content_text.split(nickname_strings[0])[1].split(' to ')[0][1:]
	elif nickname_strings[1] in content_text or nickname_strings[2] in content_text:
		nickname = content_text.split('nickname')[1].split(' to ')[1][:-1]
		name = message_dict['sender']

	nicknames.append({'name': name, 'nickname': nickname, 'datetime': message_dict['timestamp']})

df = pd.DataFrame(nicknames)
df = df.groupby(df.datetime.dt.date)['nickname'].apply(', '.join).to_frame()

print('Nicknames grouped by day')
display(HTML(df.to_html()))

In [20]:
#Print amount each member has set a nickname

members = dict()

for message_dict in message_list:
	content_text = message_dict['content_text']
	message_type = message_dict['message_type']
	if message_type != 'nickname':
		continue
	
	name = message_dict['sender']
	if name not in members:
		members[name] = 0
	else:
		members[name] += 1

members = sorted(members.items(), key=lambda x:x[1], reverse=True)
print('Amount each member has set or cleared a nickname')
display(members)

Amount each member has set a nickname


{'Robby Royston': 196,
 'Ben Wood': 179,
 'Daniel Kinahan': 152,
 'Alex Staszak': 93,
 'Caelan Rae-Oulette': 82,
 'Nick Cornell': 59,
 'Erik Porteous': 32,
 'Tristan Turigan': 30,
 'Theo Mohamed': 8,
 'Brye Loelke': 6,
 'Francois Jolicoeur': 2}

In [11]:
#Print amount each member has said a certain word

#Make sure no string is a substring of another in this list
keys = ['friend']

tally = dict.fromkeys(members, 0)

for message_dict in message_list:
	content_text = message_dict['content_text'].casefold()
	message_type = message_dict['message_type']
	num = 0
	
	if not content_text or message_type != 'regular':
		continue
	for key in keys:
		num += content_text.count(key)

	name = message_dict['sender']
	if name not in members:
		members[name] = num
	else:
		members[name] += num

members = sorted(members.items(), key=lambda x:x[1], reverse=True)
print('Word frequency of: ' + str(keys))
display(members)

Word frequency of: cuck


{'Robby Royston': 96,
 'Caelan Rae-Oulette': 85,
 'Daniel Kinahan': 50,
 'Ben Wood': 35,
 'Alex Staszak': 27,
 'Tristan Turigan': 26,
 'Nick Cornell': 17,
 'Erik Porteous': 15,
 'Theo Mohamed': 8,
 'Francois Jolicoeur': 0}

In [ ]:
#Print amount each word has been said

extra = {'u', 'im', 'dont', 'yea', 'thats', 'yeah', 'ill', 'oh', 'got', 'ur', 'tho', 'actually'}
stopwords = set(STOPWORDS) | extra

words = dict()

for message_dict in message_list:
	content_text = message_dict['content_text'].translate(str.maketrans('', '', string.punctuation)).casefold()
	message_type = message_dict['message_type']
	if not content_text or message_type != 'regular':
		continue

	print(content_text)
	for word in content_text.split():
		if word not in stopwords:
			if word not in words:
				words[word] = 0
			words[word] += 1

words = sorted(words.items(), key=lambda x:x[1], reverse=True)
print('Word frequencies')
display(words)

In [ ]:
#Print wordcloud
words = dict()
comment_words = ''
extra = {'u', 'im', 'dont', 'yea', 'thats', 'yeah', 'ill', 'oh', 'got', 'ur', 'tho', 'actually'}
stopwords = set(STOPWORDS) | extra

for message_dict in message_list:
	content_text = message_dict['content_text'].translate(str.maketrans('', '', string.punctuation)).casefold()
	message_type = message_dict['message_type']
	if not content_text or message_type != 'regular':
		continue

	comment_words += content_text + " "
print(comment_words)

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.pyplot.figure(figsize = (8, 8), facecolor = None)
plt.pyplot.imshow(wordcloud, interpolation="bilinear")
plt.pyplot.axis("off")
plt.pyplot.tight_layout(pad = 0)
 
plt.pyplot.show()

In [ ]:
# Print kickee and kicker amounts for each guy

kickers = dict()
kickees = dict()

for message_dict in message_list:
	content_text = message_dict['content_text']
	name = message_dict['sender']
	if message_dict['message_type'] != 'group':
		continue
	if 'removed' in content_text:
		#Crystal cowboy removed Nick Cornell from the group.
		if name not in kickers:
			kickers[name] = 1
		else:
			kickers[name] += 1

		kickee = content_text.split(' removed ')[1].replace(' from the group.', '')
		if kickee == 'you':
			kickee = 'Daniel Kinahan'

		if kickee not in kickees:
			kickees[kickee] = 1
		else:
			kickees[kickee] += 1

        
kickers = sorted(kickers.items(), key=lambda x:x[1], reverse=True)
kickees = sorted(kickees.items(), key=lambda x:x[1], reverse=True)

print('Kickers')
display(kickers)

print('Kickees')
display(kickees)

In [ ]:
# Print unique kick connections

kicks = dict()

for message_dict in message_list:
	content_text = message_dict['content_text']
	name = message_dict['sender']
	if message_dict['message_type'] != 'group':
		continue
	if 'removed' in content_text:
		#Crystal cowboy removed Nick Cornell from the group.

		kickee = content_text.split(' removed ')[1].replace(' from the group.', '')
		if kickee == 'you':
			kickee = 'Daniel Kinahan'

		kick = name + " removed " + kickee
		if kick in kicks:
			kicks[kick] += 1
		else:
			kicks[kick] = 1

        
kicks = sorted(kicks.items(), key=lambda x:x[1], reverse=True)

print('Kicks')
display(kicks)

In [ ]:
message_count = dict()

for message_dict in message_list:
    name = message_dict['sender']
    if name not in message_count:
        message_count[name] = 0
    message_count[name] += 1

message_count = sorted(message_count.items(), key=lambda x:x[1], reverse=True)

print('Message count:')
display(message_count)

In [ ]:
#Print message frequency over time per person

plt.rcParams["figure.figsize"] = (16,5)

df = pd.DataFrame(message_list)
# df.groupby([df.datetime.dt.year, df.datetime.dt.month])['sender'].count().plot()

members = ['Robby Royston', 'Erik Porteous', 'Nick Cornell', 'Tristan Turigan', 'Daniel Kinahan', 'Alex Staszak', 'Ben Wood', 'Caelan Rae-Oulette', 'Theo Mohamed', 'Francois Jolicoeur']
for member in members:
    temp=df[df['sender']==member]
    ax = temp.groupby(df.timestamp.dt.to_period('M'))['sender'].count().plot(x_compat=True)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
ax.legend(members)